### HVS Orbit Integration in a Dynamic MW+LMC Potential
This notebook performs a backward orbit integration for a single Hypervelocity Star (HVS) using a time-varying gravitational potential that accounts for the motion of both the Milky Way (MW) and the Large Magellanic Cloud (LMC). The primary goal is to compare the trajectory calculated in this more realistic, dynamic potential against the trajectory calculated in a simpler, static potential.

* **Data Loading:**
    * Loads the pre-processed 6D Cartesian phase-space data for the HVS sample.
    * Loads the center-of-mass orbital trajectories for both the MW and LMC, as derived from the Garavito-Camargo et al. (2019) simulations.

* **Potential & Integration:**
    * Imports the custom `MWPotential` class, which has been configured to use the MW and LMC orbital data to create a time-dependent potential.
    * Integrates the orbit of the HVS backward in time for 400 Myr using leapfrog integrator that handles the time-varying acceleration.

* **Comparison & Visualization:**
    * For comparison, it also integrates the same HVS orbit using Gala's built-in static `MilkyWayPotential`.
    * Generates a series of plots to visually compare the results from the two models:
        1.  A 3D plot of the orbital trajectories.
        2.  A set of 2D projections of the orbits (XY, XZ, YZ planes).
        3.  A plot of the total energy over time to check for conservation and compare the behavior of the two potentials.

In [3]:
# library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
import os
from tqdm.notebook import tqdm

# astropy and gala imports
from astropy import constants as const
import astropy.units as u
import gala.potential as gp
import gala.dynamics as gd
import gala.integrate as gi

# package imports
script_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(script_dir, os.pardir))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

from hvs_orbital_kinematics.potentials import MWPotential
from hvs_orbital_kinematics.integrators import leapfrog_step, leapfrog_step_time_varying

# define physical constants
G_KPC_MYR = const.G.to(u.kpc**3 / (u.Msun * u.Myr**2)).value
KM_S_TO_KPC_MYR = (u.km / u.s).to(u.kpc / u.Myr)
GYR_TO_MYR = 1000.0

In [4]:
# load hvs data
hvs_data_path = os.path.join(project_root, 'data', 'processed', '6d_cartesian_data.csv')
cartesian_df = pd.read_csv(hvs_data_path)
print(f"Successfully loaded HVS data from {hvs_data_path}")

# load mw and lmc trajectory dataframes
mw_orbit_path = os.path.join(project_root, 'data', 'raw', 'trajectories', 'GC21M2b1_orbit_mw.txt')
lmc_orbit_path = os.path.join(project_root, 'data', 'raw', 'trajectories', 'GC21M3b1_orbit_lmc.txt')

# assign column names
col_names = ['time', 'x', 'y', 'z', 'vx', 'vy', 'vz']
mw_orbit_df = pd.read_csv(mw_orbit_path, delim_whitespace=True, header=None, names=col_names)
lmc_orbit_df = pd.read_csv(lmc_orbit_path, delim_whitespace=True, header=None, names=col_names)

# convert units to match integrator (Gyr -> Myr)
mw_orbit_df['time'] *= GYR_TO_MYR
lmc_orbit_df['time'] *= GYR_TO_MYR
print("Successfully loaded and processed MW and LMC orbital data.")

Successfully loaded HVS data from /mnt/c/Users/Andrew Qin/Desktop/HVS/data/processed/6d_cartesian_data.csv
Successfully loaded and processed MW and LMC orbital data.


/tmp/ipykernel_683945/3188514946.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  mw_orbit_df = pd.read_csv(mw_orbit_path, delim_whitespace=True, header=None, names=col_names)
/tmp/ipykernel_683945/3188514946.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  lmc_orbit_df = pd.read_csv(lmc_orbit_path, delim_whitespace=True, header=None, names=col_names)
